In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import re

In [2]:
sparql = SPARQLWrapper("http://localhost:9999/blazegraph/namespace/hogeraad/sparql")

In [3]:
def create_link(ecli_from, ecli_to, sparql):
    base_query = '''
    prefix dcterm: <http://purl.org/dc/terms/>
    insert data 
    {{<http://deeplink.rechtspraak.nl/uitspraak?id={}>
        dcterm:references 
        <http://deeplink.rechtspraak.nl/uitspraak?id={}> 
    .}}'''
    query = base_query.format(ecli_from, ecli_to)
    sparql.method = 'POST'
    sparql.setQuery(query)
    res = sparql.query()

In [4]:
query = '''
prefix dcterm: <http://purl.org/dc/terms/>
select ?id ?name
where {
    ?id dcterm:hasVersion ?name.
    filter (?name != "Rechtspraak.nl").
  }
'''

sparql.setQuery(query)
sparql.setReturnFormat(JSON)
ret = sparql.query()
id_name = ret.convert()

In [5]:
id_name['results']['bindings'][10]

{'id': {'type': 'uri',
  'value': 'http://deeplink.rechtspraak.nl/uitspraak?id=ECLI:NL:HR:2007:BA7217'},
 'name': {'type': 'literal',
  'value': 'JA 2007/157 met annotatie van W. Bouman'}}

In [6]:
len(id_name['results']['bindings'])

81208

In [7]:
regex = r'(.*)( met annotatie van .*$)'

aliases = {}

for item in id_name['results']['bindings']:
    id0 = item['id']['value']
    name = item['name']['value']
    # If the name contains an annotation, remove that one
    match = re.match(regex, name)
    if match is not None:
        name = match.groups()[0]
    
    aliases[name] = id0

In [19]:
list([(re.match('NJ ([0-9]*)[^0-9]{1,2}([0-9]*)', k).groups(), k) for k in aliases.keys() if re.match('NJ ', k) is not None])[:10]

[(('2009', '250'), 'NJ 2009, 250'),
 (('2006', '339'), 'NJ 2006, 339'),
 (('2012', '409'), 'NJ 2012/409'),
 (('2006', '36'), 'NJ 2006, 36'),
 (('2002', '198'), 'NJ 2002, 198'),
 (('2007', '529'), 'NJ 2007, 529'),
 (('2000', '330'), 'NJ 2000, 330'),
 (('2007', '243'), 'NJ 2007, 243'),
 (('2014', '447'), 'NJ 2014/447'),
 (('2004', '576'), 'NJ 2004, 576')]

In [40]:

match = re.match(regex, 'Jaea e 32351')
match.groups()

AttributeError: 'NoneType' object has no attribute 'groups'

In [30]:
regex = r'(.*)(met annotatie van W. Bouman)'
match = re.match(regex, ex_name)

('JA 2007/157 ', 'met annotatie van W. Bouman')